<a href="https://colab.research.google.com/github/MariamTawfiq/ChatBot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#import necessary libraries
import numpy as np
import json
import random
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf

In [30]:
# Load the intents from the JSON file
with open('intents.json', 'r') as file:
    intents = json.load(file)['intents']

In [31]:
# Initialize the lemmatizer and tokenizer
lemmatizer = WordNetLemmatizer()
tokenizer = word_tokenize
# Create a list of all words in the intents, and a list of all intents
words = []
classes = []
documents = []
for intent in intents:
    for pattern in intent['patterns']:
        # Tokenize and lemmatize each word in the pattern
        words_in_pattern = tokenizer(pattern.lower())
        words_in_pattern = [lemmatizer.lemmatize(word) for word in words_in_pattern]
        # Add the words to the list of all words
        words.extend(words_in_pattern)
        # Add the pattern and intent to the list of all documents
        documents.append((words_in_pattern, intent['tag']))
        # Add the intent to the list of all intents
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [32]:
# Remove duplicates and sort the words and classes
words = sorted(list(set(words)))
classes = sorted(classes)

In [33]:
# Create training data as a bag of words
training_data = []
for document in documents:
    bag = []
    # Create a bag of words for each document
    for word in words:
        bag.append(1) if word in document[0] else bag.append(0)
    # Append the bag of words and the intent tag to the training data
    output_row = [0] * len(classes)
    output_row[classes.index(document[1])] = 1
    training_data.append([bag, output_row])

In [34]:
# Shuffle the training data and split it into input and output lists
random.shuffle(training_data)
training_data = np.array(training_data, dtype=object)
train_x = list(training_data[:, 0])
train_y = list(training_data[:, 1])

In [35]:
# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

In [36]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5)


Epoch 1/200
22/22 [==============================] - 1s 2ms/step - loss: 3.3828 - accuracy: 0.0280 
Epoch 2/200
22/22 [==============================] - 0s 2ms/step - loss: 3.3771 - accuracy: 0.0374
Epoch 3/200
22/22 [==============================] - 0s 2ms/step - loss: 3.3281 - accuracy: 0.0748
Epoch 4/200
22/22 [==============================] - 0s 2ms/step - loss: 3.2379 - accuracy: 0.1121
Epoch 5/200
22/22 [==============================] - 0s 2ms/step - loss: 3.1471 - accuracy: 0.1869
Epoch 6/200
22/22 [==============================] - 0s 2ms/step - loss: 3.1231 - accuracy: 0.1869
Epoch 7/200
22/22 [==============================] - 0s 3ms/step - loss: 3.0521 - accuracy: 0.2523
Epoch 8/200
22/22 [==============================] - 0s 3ms/step - loss: 2.9919 - accuracy: 0.1776
Epoch 9/200
22/22 [==============================] - 0s 3ms/step - loss: 2.9204 - accuracy: 0.2150
Epoch 10/200
22/22 [==============================] - 0s 2ms/step - loss: 2.7548 - accuracy: 0.3178
Epoch 11

In [39]:
# Define a function to process user input and generate a response
def get_response(user_input):
    # Tokenize and lemmatize the user input
    words_in_input = tokenizer(user_input.lower())
    words_in_input = [lemmatizer.lemmatize(word) for word in words_in_input]
    
    # Create a bag of words for the user input
    bag = [0] * len(words)
    for word in words_in_input:
        for i, w in enumerate(words):
            if w == word:
                bag[i] = 1
    
    # Predict the intent of the user input using the trained model
    results = model.predict(np.array([bag]), verbose=0)[0]
    # Get the index of the highest probability result
    index = np.argmax(results)
    # Get the corresponding intent tag
    tag = classes[index]
    
    # If the probability of the predicted intent is below a certain threshold, return a default response
    if results[index] < 0.5:
        return "I'm sorry, I don't understand. Can you please rephrase?"
    
    # Get a random response from the intent
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
    
    return response

# Main loop to get user input and generate responses
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
            break
    response = get_response(user_input)
    print("RaMona:", response)


You: hi 
RaMona: Hi there! What can I do for you?
You: can u help me
RaMona: Yes Sure, How can I support you
You: i forget my password
RaMona: okay you need to reset your password,follow my steps click on forget password below the page then enter your email and we will send a activition code so you can reset your password
You: okai thanks
RaMona: My pleasure
You: bye
RaMona: Have a nice day
You: quit
